
# Reading SNR Files (GNSS‑IR)

This notebook starts the **data** portion of the course. You'll load SNR time series, inspect
basic quality, and make the canonical **SNR vs elevation** plots for selected satellites (PRNs)
and bands (L1/L2/L5).



## Expected Input Formats

### UNAVCO‑style SNR text (`.snr`, `.txt`, `.csv`)
Columns (order may vary; extra columns are ignored):
```
time, prn, elev_deg, az_deg, snr_l1, snr_l2, snr_l5
```
- `time`: ISO or GPS time string
- `prn`: Gxx/Rxx/Exx
- `elev_deg`: elevation (deg), `az_deg`: azimuth (deg, clockwise from north)
- `snr_l*`: SNR or C/N0 (dB‑Hz or normalized units)


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path



## Robust Parser


In [ ]:

def _normalize(name: str) -> str:
    return ''.join(ch for ch in name.lower() if ch.isalnum())

KEYMAP = {
    'time': 'time','epoch':'time','datetime':'time',
    'prn':'prn','sat':'prn','satellite':'prn',
    'elev':'elev_deg','elevation':'elev_deg','elevdeg':'elev_deg',
    'az':'az_deg','azimuth':'az_deg','azdeg':'az_deg',
    'snr':'snr','snrl1':'snr_l1','snrl2':'snr_l2','snrl5':'snr_l5',
    'cn0':'snr','cn0l1':'snr_l1','cn0l2':'snr_l2','cn0l5':'snr_l5',
}

def read_snr_text(path: str) -> pd.DataFrame:
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(path)
    with path.open('r', encoding='utf-8', errors='ignore') as f:
        first = f.readline()
    sep = ',' if (first.count(',') >= 1) else None
    df = pd.read_csv(path, sep=sep, comment='#', engine='python')
    colmap = {}
    for c in df.columns:
        norm = _normalize(str(c))
        if norm in KEYMAP:
            out = KEYMAP[norm]
        else:
            if norm.startswith('snr') or norm.startswith('cn0'):
                if 'l1' in norm: out = 'snr_l1'
                elif 'l2' in norm: out = 'snr_l2'
                elif 'l5' in norm: out = 'snr_l5'
                else: out = 'snr'
            else:
                out = None
        if out:
            if out in colmap.values():
                k = 2
                while f'{out}_{k}' in colmap.values():
                    k += 1
                out = f'{out}_{k}'
            colmap[c] = out
    df = df.rename(columns=colmap)
    keep = [c for c in df.columns if c in {'time','prn','elev_deg','az_deg','snr','snr_l1','snr_l2','snr_l5'}]
    df = df[keep]
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'], errors='coerce')
    for k in ['elev_deg','az_deg','snr','snr_l1','snr_l2','snr_l5']:
        if k in df.columns:
            df[k] = pd.to_numeric(df[k], errors='coerce')
    if 'elev_deg' in df.columns:
        df = df.dropna(subset=['elev_deg'])
    if 'prn' in df.columns:
        df['prn'] = df['prn'].astype(str).str.strip()
    return df.reset_index(drop=True)



## Rising vs Setting Tag


In [ ]:

def tag_rising_setting(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    if 'time' not in out.columns:
        out['arc_type'] = 'unknown'
        return out
    out = out.sort_values(['prn','time'])
    out['dE_dt'] = np.nan
    for prn, g in out.groupby('prn'):
        idx = g.index
        t = g['time'].astype('int64')/1e9
        e = g['elev_deg'].values
        dt = np.gradient(t)
        de = np.gradient(e)
        dEdt = np.divide(de, dt, out=np.zeros_like(de), where=dt!=0)
        out.loc[idx, 'dE_dt'] = dEdt
    out['arc_type'] = np.where(out['dE_dt']>=0, 'rising', 'setting')
    return out



## Plot Helper


In [ ]:

def plot_snr_vs_elev(df, prn='G12', band='snr_l1', elev_min=5.0, elev_max=30.0, arc='both'):
    if band not in df.columns:
        raise ValueError(f'Band {band} not in dataframe columns: {list(df.columns)}')
    dff = df.copy()
    if 'arc_type' in dff.columns and arc in ('rising','setting'):
        dff = dff[dff['arc_type']==arc]
    dff = dff[(dff['prn']==prn) & (dff['elev_deg']>=elev_min) & (dff['elev_deg']<=elev_max)]
    if dff.empty:
        print('No data after filters.')
        return
    plt.figure(figsize=(7,4.5))
    plt.scatter(dff['elev_deg'], dff[band], s=8, alpha=0.8)
    plt.xlabel('Elevation (deg)')
    plt.ylabel(f'{band} (dB-Hz or rel.)')
    ttl_arc = arc if arc in ('rising','setting') else 'rising+setting'
    plt.title(f'SNR vs Elevation — {prn}, {band}, {ttl_arc}')
    plt.grid(True)
    plt.show()



## Synthetic Demo (if you don't have a file yet)


In [ ]:

def synth_snr_dataset():
    rng = pd.date_range('2024-01-01 12:00:00', periods=240, freq='30s')
    rows = []
    lam = 0.1903  # ~L1
    h = 2.0
    rho = 0.5
    phi_r = np.pi
    for prn in ['G12','G19']:
        elev = np.concatenate([np.linspace(5,30,120), np.linspace(30,5,120)])
        az = 60.0 if prn=='G12' else 140.0
        for t, E in zip(rng, elev):
            dphi = (4*np.pi*h/lam)*np.sin(np.deg2rad(E))
            snr = 45 + 10*np.log10(1 + rho**2 + 2*rho*np.cos(dphi+phi_r))
            rows.append({'time': t, 'prn': prn, 'elev_deg': E, 'az_deg': az, 'snr_l1': snr})
    df = pd.DataFrame(rows)
    return tag_rising_setting(df)

demo_df = synth_snr_dataset()
demo_df.head()



### Demo Plots


In [ ]:

plot_snr_vs_elev(demo_df, prn='G12', band='snr_l1', arc='rising')
plot_snr_vs_elev(demo_df, prn='G12', band='snr_l1', arc='setting')
plot_snr_vs_elev(demo_df, prn='G19', band='snr_l1', arc='both')



## Load Your File


In [ ]:

path = '/mnt/data/your_snr_file.snr'  # change me

try:
    df = read_snr_text(path)
    df = tag_rising_setting(df)
    print(df.head())
    prn_default = df['prn'].iloc[0] if 'prn' in df.columns and len(df)>0 else 'G12'
    band_candidates = [b for b in ['snr_l1','snr_l2','snr_l5','snr'] if b in df.columns]
    band_default = band_candidates[0] if band_candidates else 'snr_l1'
except Exception as e:
    print('Could not load file:', e)
    df = demo_df.copy()
    prn_default = 'G12'
    band_default = 'snr_l1'



## Quick Filtering (edit and run)


In [ ]:

PRN = prn_default
BAND = band_default
E_MIN = 5.0
E_MAX = 30.0
ARC = 'both'   # 'rising' | 'setting' | 'both'

plot_snr_vs_elev(df, prn=PRN, band=BAND, elev_min=E_MIN, elev_max=E_MAX, arc=ARC)



## Next Notebook
**`05_Filtering_and_Preprocessing.ipynb`** — Mask elevation ranges, split arcs, detrend/normalize SNR, and prep for spectral analysis in \(\sin E\)-space.
